In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Dense, Flatten, BatchNormalization, LeakyReLU, PReLU, Layer, GlobalAveragePooling2D

def ConvBlock(x, out_channels, kernel_size, use_bn=True, use_act=True, discriminator=False, **kwargs):
    x = Conv2D(out_channels, kernel_size, **kwargs)(x)
    if use_bn:
        x = BatchNormalization()(x)
    if use_act:
        x = LeakyReLU(alpha=0.2)(x) if discriminator else PReLU(shared_axes=[1, 2])(x)
    return x

def ResidualBlock(x, channels):
    res = ConvBlock(x, channels, kernel_size=3, padding='same', strides=(1, 1))
    res = ConvBlock(res, channels, kernel_size=3, padding='same', strides=(1, 1), use_act=False)
    return keras.layers.Add()([x, res])

class DepthToSpace(Layer):
    def __init__(self, block_size, **kwargs):
        super().__init__(**kwargs)
        self.block_size = block_size

    def call(self, x):
        return tf.nn.depth_to_space(x, self.block_size)

    def get_config(self):
        config = super().get_config()
        config['block_size'] = self.block_size
        return config

def UpsampleBlock(x, filters, scale):
    y = Conv2D(filters * (scale ** 2), kernel_size=3, strides=1, padding='same')(x)
    y = DepthToSpace(scale)(y)
    return PReLU(shared_axes=[1, 2])(y)
def Generator(input_shape=(64, 64, 3), res_blocks=16):
    inputs = keras.Input(shape=input_shape)
    x = ConvBlock(inputs, out_channels=64, kernel_size=9, strides=(1, 1), use_bn=False, padding='same')
    res = x
    
    for _ in range(res_blocks):
        res = ResidualBlock(res, channels=64)
        
    res = ConvBlock(res, out_channels=64, kernel_size=3, strides=(1, 1), use_act=False, padding='same')
    x = keras.layers.Add()([x, res])
    
    x = UpsampleBlock(x, filters=64, scale=2)
    x = ConvBlock(x, out_channels=3, kernel_size=9, strides=(1, 1), use_bn=False, use_act=False, padding='same')
    x = keras.layers.Activation('tanh')(x)
    return keras.Model(inputs, x, name='Generator')

def Discriminator(input_shape=(128, 128, 3), features=[128, 256, 512]):
    inputs = keras.Input(shape=input_shape)
    x = ConvBlock(inputs, out_channels=64, kernel_size=3, strides=(1, 1), use_bn=False, discriminator=True, padding='same')
    x = ConvBlock(x, out_channels=64, kernel_size=3, strides=(1, 1), discriminator=True, padding='same')
    
    for feat in features:
        x = ConvBlock(x, out_channels=feat, kernel_size=3, strides=(1, 1), discriminator=True, padding='same')
        x = ConvBlock(x, out_channels=feat, kernel_size=3, strides=(2, 2), discriminator=True, padding='same')
        
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(1)(x)
    return keras.Model(inputs, x, name='Discriminator')
